# Train.py

In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import os, sys

DATA_SIZE = 10
SAVE_PATH = './save'
EPOCHS = 10
LEARNING_RATE = 0.01
MODEL_NAME = 'test'

In [3]:
if not os.path.exists(SAVE_PATH):
    os.mkdir(SAVE_PATH)

data = (np.random.rand(DATA_SIZE, 2), np.random.rand(DATA_SIZE, 1))
test = (np.random.rand(DATA_SIZE // 8, 2), np.random.rand(DATA_SIZE // 8, 1))

In [4]:
print("Train X shape = ",np.random.rand(DATA_SIZE, 2).shape," and Train Y shape = ",np.random.rand(DATA_SIZE, 1).shape)

Train X shape =  (10, 2)  and Train Y shape =  (10, 1)


In [5]:
print("Test X shape = ",np.random.rand(DATA_SIZE // 8, 2).shape," and Test Y shape = ",np.random.rand(DATA_SIZE // 8, 1).shape)

Test X shape =  (1, 2)  and Test Y shape =  (1, 1)


In [6]:
#Clears the default graph stack and resets the global default graph.
tf.reset_default_graph()

In [7]:
#tf.placeholder Inserts a placeholder for a tensor that will be always fed.
x_input = tf.placeholder(tf.float32, shape=[None, 2], name='inputs')
y_input = tf.placeholder(tf.float32, shape=[None, 1], name='targets')

net = tf.layers.dense(x_input, 16, activation=tf.nn.relu)
net = tf.layers.dense(net, 16, activation=tf.nn.relu)
pred = tf.layers.dense(net, 1, activation=tf.nn.sigmoid, name='prediction')

loss = tf.reduce_mean(tf.squared_difference(y_input, pred), name='loss')
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

W0221 12:08:06.794437 140218148456256 deprecation.py:323] From <ipython-input-7-b95f9014f96b>:5: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0221 12:08:06.798921 140218148456256 deprecation.py:506] From /home/developer/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
#Finds the filename of latest saved checkpoint file.

#Returns - The full path to the latest checkpoint or None if no checkpoint was found.
checkpoint = tf.train.latest_checkpoint(SAVE_PATH)
should_train = checkpoint == None


In [9]:
with tf.Session() as sess:
    #Returns an Op that initializes global variables.
    sess.run(tf.global_variables_initializer())
    if should_train:
        print("Training")
        #Set up a saver to do model checkpoint
        saver = tf.train.Saver()
        for epoch in range(EPOCHS):
            
            # sess.run runs from the last node of a graph
            # here in our graph train_step and loss are the last node of a graph
            # sess.run return two results , one from train_step that is thrown away
            # one from loss that is stored in curr_loss
            
            _, curr_loss = sess.run([train_step, loss], feed_dict={x_input: data[0], y_input: data[1]})
            print('EPOCH = {}, LOSS = {:0.4f}'.format(epoch, curr_loss))
            
        #saving the model
        path = saver.save(sess, SAVE_PATH + '/' + MODEL_NAME + '.ckpt')
        print("saved at {}".format(path))
    else:
        print("Restoring")
        #getting the default graph
        graph = tf.get_default_graph()
        
        #here checkpoint stores the path where the meta graph is stored
        saver = tf.train.import_meta_graph(checkpoint + '.meta')
        
        #restoring the session from the path of checkpoint and saving it in sess variable
        saver.restore(sess, checkpoint)
        
        #getting the loss tensor which we created earlier
        loss = graph.get_tensor_by_name('loss:0')

        test_loss = sess.run(loss, feed_dict={'inputs:0': test[0], 'targets:0': test[1]})
        print(sess.run(pred, feed_dict={'inputs:0': np.random.rand(10,2)}))
        print("TEST LOSS = {:0.4f}".format(test_loss))

Training
EPOCH = 0, LOSS = 0.0770
EPOCH = 1, LOSS = 0.0723
EPOCH = 2, LOSS = 0.0683
EPOCH = 3, LOSS = 0.0655
EPOCH = 4, LOSS = 0.0629
EPOCH = 5, LOSS = 0.0607
EPOCH = 6, LOSS = 0.0587
EPOCH = 7, LOSS = 0.0567
EPOCH = 8, LOSS = 0.0551
EPOCH = 9, LOSS = 0.0538
saved at ./save/test.ckpt


In [10]:
np.random.rand(10,2)

array([[0.55044176, 0.93159502],
       [0.76612671, 0.38042123],
       [0.38452544, 0.76558362],
       [0.11171368, 0.7268584 ],
       [0.99162821, 0.14359536],
       [0.3662095 , 0.21682761],
       [0.38142034, 0.41385196],
       [0.41378229, 0.09235898],
       [0.88542082, 0.97254922],
       [0.78248991, 0.82300099]])

In [11]:
checkpoint = tf.train.latest_checkpoint(SAVE_PATH)
should_train = checkpoint
print(should_train)

./save/test.ckpt


In [13]:
with tf.Session() as sess:
    #Returns an Op that initializes global variables.
    sess.run(tf.global_variables_initializer())
    print("Restoring")
    #getting the default graph
    graph = tf.get_default_graph()

    #here checkpoint stores the path where the meta graph is stored
    saver = tf.train.import_meta_graph(checkpoint + '.meta')

    #restoring the session from the path of checkpoint and saving it in sess variable
    saver.restore(sess, checkpoint)

    #getting the loss tensor which we created earlier
    loss = graph.get_tensor_by_name('loss:0')
    
    #TEST LOSS returns just one value because test[0] and test[1] have only one row
    test_loss = sess.run(loss, feed_dict={'inputs:0': test[0], 'targets:0': test[1]})
    
    #pred is the last node of the graph for x_input
    print(sess.run(pred, feed_dict={'inputs:0': np.random.rand(10,2)}))
    print("TEST LOSS = {:0.4f}".format(test_loss))

Restoring
[[0.5898237 ]
 [0.5804689 ]
 [0.5629724 ]
 [0.6561172 ]
 [0.52727747]
 [0.656306  ]
 [0.63355345]
 [0.6177589 ]
 [0.531047  ]
 [0.6231788 ]]
TEST LOSS = 0.2909


# Serve.py

In [1]:
import tensorflow as tf
import os

SAVE_PATH = './save'
MODEL_NAME = 'test'
VERSION = 1
SERVE_PATH = './serve/{}/{}'.format(MODEL_NAME, VERSION)

#Finds the filename of latest saved checkpoint file.

#Returns - The full path to the latest checkpoint or None if no checkpoint was found.
checkpoint = tf.train.latest_checkpoint(SAVE_PATH)

tf.reset_default_graph()

In [2]:
with tf.Session() as sess:
    
    # import the saved graph
    saver = tf.train.import_meta_graph(checkpoint + '.meta')
    
    # get the graph for this session
    graph = tf.get_default_graph()
    sess.run(tf.global_variables_initializer())
    
    # get the tensors that we need
    inputs = graph.get_tensor_by_name('inputs:0')
    predictions = graph.get_tensor_by_name('prediction/Sigmoid:0')
    
    # create tensors info
    model_input = tf.saved_model.utils.build_tensor_info(inputs)           # Line-1
    model_output = tf.saved_model.utils.build_tensor_info(predictions)     # Line 2
    
    # build signature definition
    signature_definition = tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'inputs': model_input},    #model_input getting from Line 1
        outputs={'outputs': model_output},  #model_ouput getting from Line 2
        method_name= tf.saved_model.signature_constants.PREDICT_METHOD_NAME)

    builder = tf.saved_model.builder.SavedModelBuilder(SERVE_PATH)

    builder.add_meta_graph_and_variables(
        sess, [tf.saved_model.tag_constants.SERVING],
        signature_def_map={
            tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                signature_definition
        })
    # Save the model so we can serve it with a model server :)
    builder.save()


W0221 12:25:08.744048 139644469540672 deprecation.py:323] From <ipython-input-2-6e50e7395b6f>:15: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
